In [1]:
import pandas as pd
import numpy as np
import chess 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv("lichess_db_puzzle.csv")

# Display the first few rows
df.head(10)

,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1942,75,95,7530,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1405,74,96,30875,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1257,78,90,688,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1080,74,87,586,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
4,000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b -...,g6f5 d5c5 c2e4 h5g5 g7h8 g5f6,2844,104,85,246,crushing endgame long,https://lichess.org/e9AY2m5j/black#50,NaN
5,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1575,80,75,102,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN
6,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1353,75,86,627,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN
7,000aY,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...,g5e7 a5c3 b2c3 c6e7,1440,79,74,512,advantage master middlegame short,https://lichess.org/iihZGl6t#29,Benoni_Defense Benoni_Defense_Benoni-Indian_De...
8,000hf,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3...,e8f7 e2e6 f7f8 e6f7,1483,76,91,566,mate mateIn2 middlegame short,https://lichess.org/71ygsFeE/black#38,Horwitz_Defense Horwitz_Defense_Other_variations
9,000lC,3r3r/pQNk1ppp/1qnb1n2/1B6/8/8/PPP3PP/3R1R1K w ...,d1d6 d7d6 b7b6 a7b6,1408,76,94,3489,advantage hangingPiece middlegame short,https://lichess.org/vaqz2bx6#37,NaN


In [6]:
print(df['Themes'][:20])

0        crushing hangingPiece long middlegame
1                      advantage endgame short
2          advantage endgame rookEndgame short
3                   advantage middlegame short
4                        crushing endgame long
5            crushing endgame long pawnEndgame
6                   endgame mate mateIn2 short
7            advantage master middlegame short
8                mate mateIn2 middlegame short
9      advantage hangingPiece middlegame short
10                   crushing middlegame short
11    crushing endgame exposedKing long skewer
12          crushing endgame fork master short
13         crushing endgame short trappedPiece
14                  advantage middlegame short
15              advantage fork long middlegame
16                 advantage endgame pin short
17               advantage middlegame veryLong
18                  advantage endgame long pin
19     backRankMate endgame mate mateIn2 short
Name: Themes, dtype: object


In [7]:
def parse_themes(theme_string):
    if isinstance(theme_string, str):
        return theme_string.split()
sample_theme = "crushing hangingPiece long middlegame"
tags = parse_themes(sample_theme)
print(tags) 

['crushing', 'hangingPiece', 'long', 'middlegame']


In [11]:
df['Themes'].isnull().sum()

np.int64(0)

In [12]:
df['ParsedThemes'] = df['Themes'].apply(parse_themes)

df.head(10)

,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags,ParsedThemes
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1942,75,95,7530,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN,"[crushing, hangingPiece, long, middlegame]"
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1405,74,96,30875,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN,"[advantage, endgame, short]"
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1257,78,90,688,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN,"[advantage, endgame, rookEndgame, short]"
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1080,74,87,586,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...,"[advantage, middlegame, short]"
4,000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b -...,g6f5 d5c5 c2e4 h5g5 g7h8 g5f6,2844,104,85,246,crushing endgame long,https://lichess.org/e9AY2m5j/black#50,NaN,"[crushing, endgame, long]"
5,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1575,80,75,102,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN,"[crushing, endgame, long, pawnEndgame]"
6,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1353,75,86,627,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN,"[endgame, mate, mateIn2, short]"
7,000aY,r4rk1/pp3ppp/2n1b3/q1pp2B1/8/P1Q2NP1/1PP1PP1P/...,g5e7 a5c3 b2c3 c6e7,1440,79,74,512,advantage master middlegame short,https://lichess.org/iihZGl6t#29,Benoni_Defense Benoni_Defense_Benoni-Indian_De...,"[advantage, master, middlegame, short]"
8,000hf,r1bqk2r/pp1nbNp1/2p1p2p/8/2BP4/1PN3P1/P3QP1P/3...,e8f7 e2e6 f7f8 e6f7,1483,76,91,566,mate mateIn2 middlegame short,https://lichess.org/71ygsFeE/black#38,Horwitz_Defense Horwitz_Defense_Other_variations,"[mate, mateIn2, middlegame, short]"
9,000lC,3r3r/pQNk1ppp/1qnb1n2/1B6/8/8/PPP3PP/3R1R1K w ...,d1d6 d7d6 b7b6 a7b6,1408,76,94,3489,advantage hangingPiece middlegame short,https://lichess.org/vaqz2bx6#37,NaN,"[advantage, hangingPiece, middlegame, short]"


In [13]:
all_tags = [tag for themes in df['ParsedThemes'] for tag in themes]
from collections import Counter
tag_counts = Counter(all_tags)
print(tag_counts)

Counter({'short': 2542764, 'endgame': 2294763, 'middlegame': 2243593, 'crushing': 1935186, 'advantage': 1444247, 'mate': 1326242, 'long': 1198539, 'master': 665816, 'fork': 654030, 'oneMove': 625704, 'mateIn2': 581354, 'mateIn1': 578940, 'kingsideAttack': 391533, 'veryLong': 380967, 'sacrifice': 342756, 'pin': 296286, 'defensiveMove': 287422, 'advancedPawn': 277093, 'opening': 257421, 'discoveredAttack': 256331, 'rookEndgame': 238060, 'deflection': 203176, 'hangingPiece': 197090, 'quietMove': 191324, 'attraction': 166049, 'pawnEndgame': 154339, 'backRankMate': 152242, 'mateIn3': 141136, 'exposedKing': 134930, 'promotion': 106151, 'skewer': 105612, 'masterVsMaster': 68307, 'queensideAttack': 67481, 'intermezzo': 63471, 'clearance': 63084, 'trappedPiece': 62260, 'bishopEndgame': 60341, 'queenEndgame': 48781, 'zugzwang': 43542, 'equality': 42296, 'knightEndgame': 37204, 'capturingDefender': 36170, 'queenRookEndgame': 33148, 'attackingF2F7': 31972, 'doubleCheck': 23282, 'mateIn4': 20392, '